# F1 Win Prediction Project
#### Alex Boardman - BrainStation

## Data Preprocessing

### Importing Libraries and Notebook Setup

In [ ]:
# Install libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

Here you can add sections like:

- Renaming columns
- Drop Redundant Columns
- Changing Data Types
- Dropping Duplicates
- Handling Missing Values
- Handling Unreasonable Data Ranges
- Feature Engineering / Transformation

Use `assert` where possible to show that preprocessing is done.

### Rename Columns

In [2]:
# # Rename columns to snake_case
# df = clean_columns(df, replace={})

In [3]:
# # Rename columns
# columns_to_rename = {}
# df.rename(columns=columns_to_rename, inplace=True)

In [4]:
# # Verify columns are renamed
# df.columns

### Drop Redundant Columns

resultId, 
year	
race	
country	
nationality_of_circuit
driverId	
number

In [5]:
# # Check the proportion of the most frequent value in each column
# print('---- Frequency of the Mode (%) -----')
# mode_dict = {col: (df[col].value_counts().iat[0] / df[col].size * 100) for col in df.columns}
# mode_series = pd.Series(mode_dict)
# mode_series

In [ ]:
# # Show the value frequency of each column greater than the mode's threshold
# threshold = 80
# for col in mode_series[mode_series > threshold].index:
#     print(df[col].value_counts(dropna=False))
#     print()

In [ ]:
# # Drop columns (specify columns to drop)
# cols_to_drop = []
# df.drop(columns=cols_to_drop, axis=1, inplace=True)

In [ ]:
# # Verify columns dropped
# assert all(col not in df.columns for col in cols_to_drop)

In [ ]:
# # Drop columns (specify column indices to drop)
# df.drop(df.columns[a:b], axis=1, inplace=True)

In [ ]:
# # Verify columns dropped
# assert all(col not in df.columns for col in df.columns[a:b])

In [ ]:
# # Drop columns (specify columns to keep)
# cols_to_keep = []
# df = df[cols_to_keep]

In [ ]:
# # Verify columns dropped
# assert all(col in df.columns for col in cols_to_keep)

### Changing Data Types

In [ ]:
# # Convert columns to the right data types
# df[col] = df[col].astype('string')
# df[col] = df[col].astype('int')
# df[col] = pd.to_datetime(df[col], infer_datetime_format=True)

# # Convert to categorical datatype
# col_cat = ptypes.CategoricalDtype(categories=['A', 'B', 'C'], ordered=True)
# df['col_cat'] = df['col_cat'].astype(col_cat)

In [ ]:
# # Verify conversion
# assert ptypes.is_string_dtype(df[col])
# assert ptypes.is_numeric_dtype(df[col])
# cols_to_check = []
# assert all(ptypes.is_datetime64_any_dtype(df[col]) for col in cols_to_check)

### Dropping Duplicates

In [ ]:
# # Drop entirely duplicated rows
# df.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
# # Verify rows dropped
# assert df.duplicated().sum()==0

### Handling Unreasonable Data Ranges

In [ ]:
# # Drop affected rows
# df = df.loc[~((df['A'] == 0) | (df['B'] > 100))].reset_index()

In [ ]:
# # Verify rows dropped
# len(df)

### Feature Engineering / Transformation

#### constructor_points_at_stage_of_season

In [ ]:
constuctor_points_sum_df = fastest_lap_df.copy()

In [ ]:
# First, ensure 'Index', 'year', and 'constructor' are sorted in the order we want to process them
constuctor_points_sum_df = constuctor_points_sum_df.sort_values(by=['year', 'Index', 'constructor'])

# Initialize a new column for corrected constructor points
constuctor_points_sum_df['corrected_constructor_points'] = 0

# Use a temporary DataFrame to assist with the cumulative sum calculation
temp_df = constuctor_points_sum_df.groupby(['year', 'Index', 'constructor'])['points'].sum().groupby(level=[0, 2]).cumsum().reset_index()

# Merge this temporary DataFrame back to the original sorted DataFrame
# This step ensures each driver for the constructor at that Index sees the summed points on the constructor level
df_merged = pd.merge(constuctor_points_sum_df, temp_df, on=['year', 'Index', 'constructor'], how='left')

# The merged DataFrame now has an additional column with the cumulative points which needs to be renamed and checked
df_merged = df_merged.rename(columns={'points_y': 'constructor_points_at_stage_of_season', 'points_x': 'points'})

# Drop the previously incorrectly calculated column
df_merged.drop(columns=['corrected_constructor_points'], inplace=True)

In [ ]:
# Check the first few rows to see if "points_in_previous_race" has been updated
fastest_lap_df[['Index', 'driver_name', 'fastestLap_ms', 'fastest_lap_from_last_race']].head(50)

In [ ]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged[['Index', 'driver_name', 'year', 'constructor', 'points', 'constructor_points_at_stage_of_season']].head(50)

In [ ]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged[['Index', 'driver_name', 'year', 'constructor', 'points', 'constructor_points_at_stage_of_season']].tail(50)

That seems to have worked - now we have a column that has the cumulative points for the constructor at this stage of the season for each driver.

#### driver_points_at_stage_of_season

In [ ]:
driver_points_sum_df = df_merged.copy()

In [ ]:
# Initialize a new column for driver points at stage of season
driver_points_sum_df['driver_points_at_stage_of_season'] = 0

# Use a temporary DataFrame to assist with the cumulative sum calculation for drivers
temp_driver_df = driver_points_sum_df.groupby(['year', 'Index', 'driver_name'])['points'].sum().groupby(level=[0, 2]).cumsum().reset_index()

# Merge this temporary DataFrame back to the original DataFrame
# This step ensures each driver sees the summed points at that stage of the season
df_merged_with_driver_points = pd.merge(driver_points_sum_df, temp_driver_df, on=['year', 'Index', 'driver_name'], how='left')

# The merged DataFrame now has an additional column with the cumulative points which needs to be renamed and checked
df_merged_with_driver_points = df_merged_with_driver_points.rename(columns={'points_y': 'driver_points_at_stage_of_season', 'points_x': 'points'})

In [ ]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged_with_driver_points[['Index', 'year', 'driver_name', 'points', 'driver_points_at_stage_of_season']].head(50)

In [ ]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged_with_driver_points[['Index', 'year', 'driver_name', 'points', 'driver_points_at_stage_of_season']].tail(50)

That seems to have worked - now we have a column that has the cumulative points for the driver at this stage of the season for each driver.

### Other Feature Engineering Ideas

1) team_development_rank_last_year, 
2) status_finished_last_race
3) team_rank_first_race_after_major_regulation_change

In [ ]:
# # Get unique values of interested columns
# cols = []
# pd.unique(df[cols].values.ravel('k'))  # argument 'k' lists the values in the order of the cols 

In [ ]:
# # Create custom function
# # Google style docstrings
# # https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html
# def custom_function(param1: int, param2: str) -> bool:
#     """Example function with PEP 484 type annotations.

#     Args:
#         param1: The first parameter.
#         param2: The second parameter.

#     Returns:
#         The return value. True for success, False otherwise.

#     """

In [ ]:
# # Apply function to multiple columns
# cols = []
# df_updated = df.copy()
# df_updated[cols] = df_updated[cols].applymap(custom_function)

# # Create new aggregated boolean column
# df_updated['bool'] = df_updated[cols].any(axis=1, skipna=False)